In [1]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

%pip install -q transformers
%pip install -q loguru
%pip install -q openai langchain langchain[docarray] tiktoken unstructured
%pip install -q llama-index==0.10.4 chromadb loguru pypdf  sentence_transformers llama-index-embeddings-huggingface
%pip install -q pdf2image  pdfminer.six
%pip install -q pillow-heif
%pip install -q unstructured-inference
%pip install -q pikepdf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.7/226.7 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 815.9/815.9 kB 15.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 25.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 39.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.9/75.9 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 45.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 241.2/241.2 kB 24.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 215.3/215.3 kB 19.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 421.5/421.5 kB 28.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 274.7/274.7 kB 25.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━

In [2]:
import openai

from google.colab import userdata
openai.api_key = userdata.get('OPENAI_API_KEY')

In [3]:
!wget -O Baidu-Inc-Code-of-Business-Conduct-and-Ethics.pdf https://dl.dropbox.com/scl/fi/boxb3mvc0mv7d7tndmy0j/BAIDU-INC.-CODE-OF-BUSINESS-CONDUCT-AND-ETHICS.pdf?rlkey=gs378jtvi6gwzy0ncn0jx0876
FILE_NAME = "./Baidu-Inc-Code-of-Business-Conduct-and-Ethics.pdf"

--2024-02-19 08:56:48--  https://dl.dropbox.com/scl/fi/boxb3mvc0mv7d7tndmy0j/BAIDU-INC.-CODE-OF-BUSINESS-CONDUCT-AND-ETHICS.pdf?rlkey=gs378jtvi6gwzy0ncn0jx0876
Resolving dl.dropbox.com (dl.dropbox.com)... 162.125.65.15, 2620:100:6021:15::a27d:410f
Connecting to dl.dropbox.com (dl.dropbox.com)|162.125.65.15|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://dl.dropboxusercontent.com/scl/fi/boxb3mvc0mv7d7tndmy0j/BAIDU-INC.-CODE-OF-BUSINESS-CONDUCT-AND-ETHICS.pdf?rlkey=gs378jtvi6gwzy0ncn0jx0876 [following]
--2024-02-19 08:56:48--  https://dl.dropboxusercontent.com/scl/fi/boxb3mvc0mv7d7tndmy0j/BAIDU-INC.-CODE-OF-BUSINESS-CONDUCT-AND-ETHICS.pdf?rlkey=gs378jtvi6gwzy0ncn0jx0876
Resolving dl.dropboxusercontent.com (dl.dropboxusercontent.com)... 162.125.2.15, 2620:100:6021:15::a27d:410f
Connecting to dl.dropboxusercontent.com (dl.dropboxusercontent.com)|162.125.2.15|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 221928 (217K) [appli

## Similarity K

In [4]:
K = 5

## LangChain

In [5]:
from langchain.vectorstores import Chroma
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.embeddings import OpenAIEmbeddings
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema.output_parser import StrOutputParser
from langchain.retrievers.multi_query import MultiQueryRetriever
from langchain.text_splitter import SentenceTransformersTokenTextSplitter
from langchain.document_loaders import PyPDFLoader, UnstructuredPDFLoader

from typing import Any, List
from rich.pretty import pprint

def pretty_print(title: str=None, content: Any=None):
    if title is None:
        print(content)
        return
    print(title)
    pprint(content)

from IPython.display import display, HTML

In [6]:
import logging
logging.basicConfig()
logging.getLogger("langchain.retrievers.multi_query").setLevel(logging.INFO)

In [7]:
#@title load data { vertical-output: true}
loader = UnstructuredPDFLoader(FILE_NAME)
docs = loader.load()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


In [8]:
#@title splitting { vertical-output: true}
text_splitter = SentenceTransformersTokenTextSplitter()
texts = text_splitter.split_documents(docs)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [9]:
#@title indexing { vertical-output: true}
emb = OpenAIEmbeddings(openai_api_key=openai.api_key)
vec_db = Chroma.from_documents(documents=texts, embedding=emb)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.embeddings.openai.OpenAIEmbeddings` was deprecated in langchain-community 0.1.0 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


In [10]:
#@title LLM { vertical-output: true}
lc_model = ChatOpenAI(openai_api_key=openai.api_key, temperature=1.5)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.chat_models.openai.ChatOpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


In [11]:
#@title base-retriever { vertical-output: true}
base_retriever = vec_db.as_retriever(search_kwargs={"k":K})

In [12]:
#@title MultiQueryRetriever { vertical-output: true}
final_retriever = MultiQueryRetriever.from_llm(base_retriever, lc_model)

In [13]:
#@title QA chain { vertical-output: true}
tmpl = """
You are an assistant to answer a question from user with a context.

Context:
{context}

Question:
{question}

The response should be presented as a list of key points, after creating the title of the content,
    formatted in HTML with appropriate markup for clarity and organization.
"""
prompt = ChatPromptTemplate.from_template(tmpl)
chain = {"question": RunnablePassthrough(), "context": final_retriever} \
        | prompt \
        | lc_model \
        | StrOutputParser() \

result = chain.invoke("Waht is the doc talking about?")

display(HTML(result))

INFO:langchain.retrievers.multi_query:Generated queries: ['What is the content of the document?', 'What does the document cover?', "Can you give me an overview of the document's topics?"]


## Llama-Index

In [14]:
TEMPERATURE = 0.0
SIM_TOP_K = 3
RERANK_TOP_K = 3
WIN_SZ = 1024

In [95]:
import asyncio
from tqdm.asyncio import tqdm

import chromadb
from chromadb.api.models.Collection import Collection

from llama_index.core import (
    ServiceContext,
    SimpleDirectoryReader,
    StorageContext,
    VectorStoreIndex,
    get_response_synthesizer,
)
from llama_index.core import QueryBundle
from llama_index.llms.openai import OpenAI
from llama_index.core import PromptTemplate
from llama_index.core.llms.utils import LLMType
from llama_index.core.schema import NodeWithScore
from llama_index.core.indices.base import BaseIndex
from llama_index.core.retrievers import BaseRetriever
from llama_index.core.embeddings.utils import EmbedType
from llama_index.core.node_parser import SentenceSplitter
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.legacy.vector_stores import ChromaVectorStore
from llama_index.core.base.llms.types import CompletionResponse
from llama_index.core.node_parser import SentenceWindowNodeParser

from llama_index.core.retrievers import BaseRetriever
from llama_index.core import get_response_synthesizer
from llama_index.legacy.core.response.schema import RESPONSE_TYPE
from llama_index.core.response_synthesizers import BaseSynthesizer
from llama_index.core.query_engine import CustomQueryEngine, BaseQueryEngine

In [43]:
def create_service_context(llm: LLMType, embs: EmbedType) -> ServiceContext:
    node_parser = SentenceWindowNodeParser.from_defaults(
        window_size=WIN_SZ,
        window_metadata_key="window",
        original_text_metadata_key="original_text",
    )

    return ServiceContext.from_defaults(
        node_parser=node_parser,
        llm=llm,
        embed_model=embs,
    )

def create_vectors(path: str, collection_name="tmp_collection") -> Collection:
    chroma_client = chromadb.PersistentClient(path)
    # https://github.com/run-llama/llama_index/issues/6528
    return chroma_client.get_or_create_collection(collection_name)

def create_storage_context() -> StorageContext:
    path: str = "./db/LlamaIndexMultiVectorSummaryAgent"
    return StorageContext.from_defaults(
        vector_store=ChromaVectorStore(create_vectors(path=path))
    )


In [44]:
#@title load data { vertical-output: true}
input_files: List[str] = [FILE_NAME]
docs: SimpleDirectoryReader = SimpleDirectoryReader( input_files=input_files).load_data()

In [45]:
#@title LLM { vertical-output: true}
li_model = OpenAI(model="gpt-4-1106-preview", temperature=TEMPERATURE)

In [48]:
#@title splitting and indexing { vertical-output: true}
embs = "local:BAAI/bge-small-en-v1.5"

service_context: ServiceContext = (
    create_service_context(li_model, embs)
)

storage_context: StorageContext = (
    create_storage_context()
)

vector_index: BaseIndex = VectorStoreIndex.from_documents(
    docs,
    service_context=service_context,
    #storage_context=storage_context,
    show_progress=True,
)

<ipython-input-43-0e0cc94f5ebb>:8: DeprecationWarning: Call to deprecated class method from_defaults. (ServiceContext is deprecated, please use `llama_index.settings.Settings` instead.) -- Deprecated since version 0.10.0.
  return ServiceContext.from_defaults(


Parsing nodes:   0%|          | 0/10 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/160 [00:00<?, ?it/s]

In [54]:
#@title base-retriever { vertical-output: true}
base_retriever = vector_index.as_retriever(similarity_top_k=K)

In [208]:
#@title MultiQueriesRetriever { vertical-output: true}
class MultiQueriesRetriever(BaseRetriever):
    def __init__(self, base_retriever: BaseRetriever, model:OpenAI):
        self.template = PromptTemplate("""You are an AI language model assistant. Your task is to generate Five
    different versions of the given user question to retrieve relevant documents from a vector
    database. By generating multiple perspectives on the user question, your goal is to help
    the user overcome some of the limitations of the distance-based similarity search.
    Provide these alternative questions seperated by newlines only.

    For example, these alternative questions:

    'What is Bill Gates known for?'
│   "Can you provide information about Bill Gates' background?"

    Not:

    '1. What is Bill Gates known for?'
│   "2. Can you provide information about Bill Gates' background?"

    Original question: {question}""")
        self._retrievers = [base_retriever]
        self.base_retriever = base_retriever
        self.model = model

    @classmethod
    def flatten(cls, lst: List[List[Any]]) -> List[Any]:
        return [element for sublist in lst for element in sublist]

    def gen_queries(self, query: str) -> List[str]:
        gen_queries_model = OpenAI(model="gpt-3-turbo", temperature=1.5)
        prompt = self.template.format(question=query)
        res = self.model.complete(prompt)
        return res.text.split("\n")

    async def run_gen_queries(self,generated_queries: List[str]) -> List[NodeWithScore]:
        tasks = list(map(lambda q: self.base_retriever.aretrieve(q), generated_queries))
        res = await tqdm.gather(*tasks)
        return MultiQueriesRetriever.flatten(res)

    def _retrieve(self, query_bundle: QueryBundle) -> List[NodeWithScore]:
        return list()

    async def _aretrieve(self, query_bundle: QueryBundle) -> List[NodeWithScore]:
        query: str= query_bundle.query_str
        generated_queries: List[str] = self.gen_queries(query)
        pretty_print("generated_queries", generated_queries)
        node_with_scores = await self.run_gen_queries(generated_queries)
        node_with_scores_uniqued = dict()
        # Simplely removing duplicated nodes in this notebook.
        # For Fusion with ranking, ref:https://plg.uwaterloo.ca/~gvcormac/cormacksigir09-rrf.pdf
        node_with_scores_uniqued = {node_with_score.get_content(): node_with_score for node_with_score in node_with_scores}
        return node_with_scores_uniqued.values()

mr = MultiQueriesRetriever(base_retriever, li_model)

In [209]:
#@title { vertical-output: true}
ls = mr.gen_queries("Who is Bill Gates?")
pretty_print("ls", ls)
rls = await mr.run_gen_queries(ls)
pretty_print("rls", rls)

ls


[
│   'What are the notable achievements of Bill Gates?',
│   "Could you tell me about Bill Gates' contributions to technology and philanthropy?",
│   'In what ways has Bill Gates impacted the tech industry?',
│   'What is the biography of Bill Gates?',
│   "Can you give me a summary of Bill Gates' career and personal life?"
]

100%|██████████| 5/5 [00:00<00:00, 25.58it/s]


rls


[
│   NodeWithScore(
│   │   node=TextNode(
│   │   │   id_='697c320a-0b68-4fa6-85f4-00b6c2943957',
│   │   │   embedding=None,
│   │   │   metadata={
│   │   │   │   'window': "Employees unless the personal actions and behaviors have a negative \nimpact on the Employee's work performance or the reputation and \nlegitimate commercial interests of the Company.  \n o Open Communication  \nThe Company encourages open and honest communication about different \nviews.  \n o Valuing Employees a s Important Assets  \nThe Company believes its Employees are important assets and keys to its \nsuccess.  The Company is committed to establishing a fair and equal \nevaluation and incentive mechanism to maintain a stable and highly efficient \nworking environment.  The Company strives to maintain its success by \nproviding opportunities that enable Employees to realize their values and \ncareers.  \n o Conflicts of Interest  \nThe Company expects that each Employee will use his/her best efforts to \ndischarge his/her duty in an honest and upright manner.  This requires each \nEmployee to avoid conflicts of interest between his/her personal interest \nand the interest of the Company.  A conflict of interest occurs when an \nEmployee's private interest interferes, or appears to interfere, in any way \nwith the interests of the Company as a whole.  Employees should actively \navoid any private interest that may influence their ability to act in the \ninterests of the Company or that may make it difficult to perform their work \nobjectively and effectively.  For detailed provisions regarding conflicts of \ninterest, please refer to Baidu  Avoiding Conflicts of Interest  Policy .  Besides \nstipulated prohibitions on employees in Baidu Avoiding Conflicts of Interest  \nPolicy , Baidu.com, Inc., as a Nasdaq-listed company, is prohibited from \ndirectly or indirectly, including through any subsidiary, extending or \nmaintaining credit in the form of a personal loan to or for any of its directors \nor officers (or equivalent thereof)  \n \n\uf0a7 4.3 Illegal or Improper A ctions   \nEmployees are prohibited to conduct illegal or improper actions, which include, \nwithout limitation:  \no any action that violates applicable laws or public interests;  \no engagement in malpractice or dereliction of duties;  \no failure to discharge responsibilit y, duty and obligation;  \no failure to disclose ancillary agreements or private agreements entered into \nwith an agent, client, supplier or partner;  \no misrepresentations regarding personal experience or personal physical and \nmedical conditions to the Company; and  ",
│   │   │   │   'original_text': 'The Company strives to maintain its success by \nproviding opportunities that enable Employees to realize their values and \ncareers.  \n',
│   │   │   │   'page_label': '3',
│   │   │   │   'file_name': 'Baidu-Inc-Code-of-Business-Conduct-and-Ethics.pdf',
│   │   │   │   'file_path': 'Baidu-Inc-Code-of-Business-Conduct-and-Ethics.pdf',
│   │   │   │   'file_type': 'application/pdf',
│   │   │   │   'file_size': 221928,
│   │   │   │   'creation_date': '2024-02-19',
│   │   │   │   'last_modified_date': '2024-02-19',
│   │   │   │   'last_accessed_date': '2024-02-19'
│   │   │   },
│   │   │   excluded_embed_metadata_keys=[
│   │   │   │   'file_name',
│   │   │   │   'file_type',
│   │   │   │   'file_size',
│   │   │   │   'creation_date',
│   │   │   │   'last_modified_date',
│   │   │   │   'last_accessed_date',
│   │   │   │   'window',
│   │   │   │   'original_text'
│   │   │   ],
│   │   │   excluded_llm_metadata_keys=[
│   │   │   │   'file_name',
│   │   │   │   'file_type',
│   │   │   │   'file_size',
│   │   │   │   'creation_date',
│   │   │   │   'last_modified_date',
│   │   │   │   'last_accessed_date',
│   │   │   │   'window',
│   │   │   │   'original_text'
│   │   │   ],
│   │   │   relationships={
│   │   │   │   <NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(
│   │   │   │   │   node

In [210]:
#@title QA query { vertical-output: true}
query_text = """What is the topic of content?

    The response should be presented as a list of key points, after creating the title of the content,
    formatted in HTML with appropriate markup for clarity and organization.
    """
final_res = await RetrieverQueryEngine(mr).aquery(query_text)
display(HTML(final_res.response))

generated_queries


[
│   'What is the subject matter of the material?',
│   'Could you describe the central theme of the content?',
│   'What are the main points addressed in the content?',
│   'Can you summarize the primary focus of the content?',
│   'What is the content primarily concerned with?'
]

100%|██████████| 5/5 [00:00<00:00, 26.67it/s]


In [214]:
#@title MultiQueriers { vertical-output: true}
class MultiQueriers:
    def __init__(self,
                 base_retriever: BaseRetriever,
                 base_query_engine: BaseQueryEngine,
                 model:OpenAI,
                 sub_queries_in_bundle_to_answer: bool=True):
        self.base_retriever = base_retriever
        self.base_query_engine = base_query_engine
        self.model = model
        self.sub_queries_in_bundle_to_answer = sub_queries_in_bundle_to_answer
        self.gen_q_template = PromptTemplate("""You are an AI language model assistant. Your task is to generate Five
    different versions of the given user question to retrieve relevant documents from a vector
    database. By generating multiple perspectives on the user question, your goal is to help
    the user overcome some of the limitations of the distance-based similarity search.
    Provide these alternative questions seperated by newlines only.

    For example, these alternative questions:

    'What is Bill Gates known for?'
│   "Can you provide information about Bill Gates' background?"

    Not:

    '1. What is Bill Gates known for?'
│   "2. Can you provide information about Bill Gates' background?"

    Original question: {question}""")
        self.qa_prompt_template = PromptTemplate("""Here is the question you need to answer:

    \n --- \n {query_str} \n --- \n

    Here is any available background question + answer pairs:

    \n --- \n {q_a_pairs} \n --- \n

    Here is additional context relevant to the question:

    \n --- \n {context_str} \n --- \n

    Use the above context and any background question + answer pairs to answer the question: \n {query_str}
    """)

    def gen_queries(self, query: str) -> List[str]:
        gen_queries_model = OpenAI(model="gpt-3-turbo", temperature=1.5)
        prompt = self.gen_q_template.format(question=query)
        res = self.model.complete(prompt)
        return res.text.split("\n")

    def query_by_retriever(self, query_str: str) ->str:
        nodes = self.base_retriever.retrieve(query_str)
        res = "\n\n".join([n.node.get_content() for n in nodes])
        return res

    async def run_gen_queries(self,generated_queries: List[str], origin_query: str=None) -> str:
        sub_query_qa_pairs = list()
        if self.sub_queries_in_bundle_to_answer:
            # Answer all queries in one bundle.
            tasks = list(map(lambda q: self.base_query_engine.aquery(q), generated_queries))
            res = await tqdm.gather(*tasks)
            for idx, (query, answer) in enumerate(zip(generated_queries, res)):
                qa_pair = f"Question {idx}: {query}\nAnswer: {answer}\n"
                sub_query_qa_pairs.append(qa_pair)
            pretty_print("sub_query_qa_pairs", sub_query_qa_pairs)
            return "\n\n".join(sub_query_qa_pairs)
        else:
            # Answer queries in step down.
            # One sub-query will be answer based on the context of sub-query,
            # history of pairs of previous queries and answers.
            for idx, query in enumerate(generated_queries):
                pretty_print(f"{idx}. query", query)
                pretty_print(f"{idx}. sub_query_qa_pairs", sub_query_qa_pairs)
                context_str = self.query_by_retriever(query)
                sub_query = self.qa_prompt_template.format(
                                  query_str=query,
                                  q_a_pairs="\n\n".join(sub_query_qa_pairs),
                                  context_str=context_str
                )
                pretty_print("sub_query", sub_query)
                answer: str = self.model.complete(sub_query)
                qa_pair = f"Question {idx}: {query}\nAnswer: {answer}\n"
                sub_query_qa_pairs.append(qa_pair)
            return "\n\n".join(sub_query_qa_pairs)

    def query(self, query_str: str) -> CompletionResponse:
        return ""

    async def aquery(self, query_str: str) -> CompletionResponse:
        generated_queries: List[str] = self.gen_queries(query_str)
        sub_query_qa_pairs: str = await self.run_gen_queries(generated_queries, query_str)
        context_str = self.query_by_retriever(query_str)
        final_query: str = self.qa_prompt_template.format(query_str=query_str,
                                q_a_pairs=sub_query_qa_pairs,
                                context_str=context_str)
        pretty_print("final_query", final_query)
        response: str = self.model.complete(final_query)
        return response


In [215]:
#@title QA query, sub-queries in bundle to answer{ vertical-output: true}
query_text = """What is the topic of content?

    The response should be presented as a list of key points, after creating the title of the content,
    formatted in HTML with appropriate markup for clarity and organization.
    """
mqe=MultiQueriers(base_retriever,  vector_index.as_query_engine(), li_model, sub_queries_in_bundle_to_answer=True)
final_res = await mqe.aquery(query_text)
display(HTML(final_res.text))

100%|██████████| 5/5 [00:04<00:00,  1.22it/s]

sub_query_qa_pairs


[
│   "Question 0: What is the subject matter of the material?\nAnswer: The subject matter of the material is Baidu Inc.'s Code of Business Conduct and Ethics, which includes a summary of the company's employment practices and a description of the company's assets.\n",
│   "Question 1: Could you specify the theme of the content?\nAnswer: The theme of the content revolves around the importance of employees to the company's success and the conduct expected of employees when interacting with customers or potential customers.\n",
│   "Question 2: What is the central focus of the content in question?\nAnswer: The central focus of the content in question is on the importance of employees to the company's success and the need for vigilance regarding financial integrity and adherence to established review and approval procedures.\n",
│   "Question 3: Can you describe the main idea of the content?\nAnswer: The main idea of the content is that the company, Baidu Inc., considers its employees to be valuable assets and crucial to its success. Additionally, there is an intention to summarize the company's employment practices.\n",
│   "Question 4: What is the content primarily about?\nAnswer: The content primarily discusses the principles and values related to employee appreciation and reputation management within a company. It emphasizes the importance of employees as key assets to the company's success and acknowledges the impact of media and the investment community on the company's reputation.\n"
]

final_query


"Here is the question you need to answer:\n\n    \n --- \n What is the topic of content?\n\n    The response should be presented as a list of key points, after creating the title of the content,\n    formatted in HTML with appropriate markup for clarity and organization.\n     \n --- \n\n\n    Here is any available background question + answer pairs:\n\n    \n --- \n Question 0: What is the subject matter of the material?\nAnswer: The subject matter of the material is Baidu Inc.'s Code of Business Conduct and Ethics, which includes a summary of the company's employment practices and a description of the company's assets.\n\n\nQuestion 1: Could you specify the theme of the content?\nAnswer: The theme of the content revolves around the importance of employees to the company's success and the conduct expected of employees when interacting with customers or potential customers.\n\n\nQuestion 2: What is the central focus of the content in question?\nAnswer: The central focus of the content in question is on the importance of employees to the company's success and the need for vigilance regarding financial integrity and adherence to established review and approval procedures.\n\n\nQuestion 3: Can you describe the main idea of the content?\nAnswer: The main idea of the content is that the company, Baidu Inc., considers its employees to be valuable assets and crucial to its success. Additionally, there is an intention to summarize the company's employment practices.\n\n\nQuestion 4: What is the content primarily about?\nAnswer: The content primarily discusses the principles and values related to employee appreciation and reputation management within a company. It emphasizes the importance of employees as key assets to the company's success and acknowledges the impact of media and the investment community on the company's reputation.\n \n --- \n\n\n    Here is additional context relevant to the question: \n\n    \n --- \n If an Employee receives a request for information from any \nsecurities mark et professionals or shareholders, he or she should promptly \ncontact the Investor Relations Manager to coordinate a response to such \nrequest.  \n\n\no Valuing Employees a s Important Assets  \nThe Company believes its Employees are important assets and keys to its \nsuccess. \n\nThe following is intended to be a summary of the Company's employment practices. \n\n\nParticular attention should be paid to:  \no financial results that seem inconsistent with the performance of the \nunderlying business;  \no transactions that do not seem to have an obvious business purpose; and  \no requests to circumvent ordinary review and approval procedures.  \n\n\nWhat is written or \nsaid about the Company in the news media and investment community directly \nimpacts its reputation, positively or negatively.  \n --- \n\n\n    Use the above context and any background question + answer pairs to answer the question: \n What is the topic of content?\n\n    The response should be presented as a list of key points, after creating the title of the content,\n    formatted in HTML with appropriate markup for clarity and organization.\n    \n    "

In [216]:
#@title QA query, sub-queries step down to generate final answer { vertical-output: true}
query_text = """What is the topic of content?

    The response should be presented as a list of key points, after creating the title of the content,
    formatted in HTML with appropriate markup for clarity and organization.
    """
mqe=MultiQueriers(base_retriever,  vector_index.as_query_engine(), li_model, sub_queries_in_bundle_to_answer=False)
final_res = await mqe.aquery(query_text)
display(HTML(final_res.text))

0. query


'What subject matter does the content address?'

0. sub_query_qa_pairs


[]

sub_query


...

[
│   "Question 0: What subject matter does the content address?\nAnswer: The content addresses the subject matter of handling requests for information about a company from securities market professionals, shareholders, and members of the media. It outlines the company's policy for employees to refer such inquiries to the Investor Relations Manager to ensure that responses are coordinated and that the company provides timely, accurate, and complete information to the public. Additionally, it mentions that the content is a summary of the company's employment practices, which likely includes the protocol for communication with external parties.\n",
│   "Question 1: Could you specify the central theme of the content?\nAnswer: The central theme of the content is the importance of responsible and ethical conduct by employees in their interactions with external parties, with a focus on compliance with the company's policies and the avoidance of conflicts of interest. It emphasizes the value the company places on its employees as key assets and outlines the need for employees to seek guidance when in doubt about the legality or compliance of their actions. The content also highlights the company's commitment to coordinated communication with securities market professionals, shareholders, and the media through the Investor Relations Manager to ensure the provision of timely, accurate, and complete information.\n",
│   "Question 2: What is the main focus of the content described?\nAnswer: The main focus of the content described is the company's employment practices, particularly regarding the ethical conduct and responsibilities of employees when dealing with external inquiries and ensuring the integrity of financial information. It emphasizes the importance of employees as valuable assets to the company's success and outlines the need for adherence to the company's Code of Conduct, especially in relation to coordinated communication with securities market professionals, shareholders, and the media through the designated Investor Relations Manager. The content also stresses the importance of vigilance against any financial inconsistencies, transactions without clear business purposes, and attempts to bypass standard review and approval procedures, all of which are crucial for maintaining the company's reputation.\n",
│   "Question 3: Can you identify the primary subject covered in the content?\nAnswer: The primary subject covered in the content is the company's employment practices related to the ethical conduct and responsibilities of employees when responding to inquiries from securities market professionals, shareholders, and members of the media. It includes the policy of referring such inquiries to the Investor Relations Manager to ensure coordinated, accurate, and complete communication. The content also touches on the importance of making fair and accurate comments about competitors based on publicly available information, and it advises employees to seek guidance when in doubt about the legality or compliance with company policies. Additionally, it relates to the adherence to the company's Code of Conduct and the maintenance of the company's reputation through vigilance against financial inconsistencies and adherence to standard review and approval procedures.\n"
]

sub_query


"Here is the question you need to answer:\n\n    \n --- \n What is the content primarily concerned with? \n --- \n\n\n    Here is any available background question + answer pairs:\n\n    \n --- \n Question 0: What subject matter does the content address?\nAnswer: The content addresses the subject matter of handling requests for information about a company from securities market professionals, shareholders, and members of the media. It outlines the company's policy for employees to refer such inquiries to the Investor Relations Manager to ensure that responses are coordinated and that the company provides timely, accurate, and complete information to the public. Additionally, it mentions that the content is a summary of the company's employment practices, which likely includes the protocol for communication with external parties.\n\n\nQuestion 1: Could you specify the central theme of the content?\nAnswer: The central theme of the content is the importance of responsible and ethical conduct by employees in their interactions with external parties, with a focus on compliance with the company's policies and the avoidance of conflicts of interest. It emphasizes the value the company places on its employees as key assets and outlines the need for employees to seek guidance when in doubt about the legality or compliance of their actions. The content also highlights the company's commitment to coordinated communication with securities market professionals, shareholders, and the media through the Investor Relations Manager to ensure the provision of timely, accurate, and complete information.\n\n\nQuestion 2: What is the main focus of the content described?\nAnswer: The main focus of the content described is the company's employment practices, particularly regarding the ethical conduct and responsibilities of employees when dealing with external inquiries and ensuring the integrity of financial information. It emphasizes the importance of employees as valuable assets to the company's success and outlines the need for adherence to the company's Code of Conduct, especially in relation to coordinated communication with securities market professionals, shareholders, and the media through the designated Investor Relations Manager. The content also stresses the importance of vigilance against any financial inconsistencies, transactions without clear business purposes, and attempts to bypass standard review and approval procedures, all of which are crucial for maintaining the company's reputation.\n\n\nQuestion 3: Can you identify the primary subject covered in the content?\nAnswer: The primary subject covered in the content is the company's employment practices related to the ethical conduct and responsibilities of employees when responding to inquiries from securities market professionals, shareholders, and members of the media. It includes the policy of referring such inquiries to the Investor Relations Manager to ensure coordinated, accurate, and complete communication. The content also touches on the importance of making fair and accurate comments about competitors based on publicly available information, and it advises employees to seek guidance when in doubt about the legality or compliance with company policies. Additionally, it relates to the adherence to the company's Code of Conduct and the maintenance of the company's reputation through vigilance against financial inconsistencies and adherence to standard review and approval procedures.\n \n --- \n\n\n    Here is additional context relevant to the question: \n\n    \n --- \n Particular attention should be paid to:  \no financial results that seem inconsistent with the performance of the \nunderlying business;  \no transactions that do not seem to have an obvious business purpose; and  \no requests to circumvent ordinary review and approval procedures.  \n\n\no Valuing Employees a s Important Assets  \nThe Company believes its Employees are important assets and keys to its \nsucce

final_query


"Here is the question you need to answer:\n\n    \n --- \n What is the topic of content?\n\n    The response should be presented as a list of key points, after creating the title of the content,\n    formatted in HTML with appropriate markup for clarity and organization.\n     \n --- \n\n\n    Here is any available background question + answer pairs:\n\n    \n --- \n Question 0: What subject matter does the content address?\nAnswer: The content addresses the subject matter of handling requests for information about a company from securities market professionals, shareholders, and members of the media. It outlines the company's policy for employees to refer such inquiries to the Investor Relations Manager to ensure that responses are coordinated and that the company provides timely, accurate, and complete information to the public. Additionally, it mentions that the content is a summary of the company's employment practices, which likely includes the protocol for communication with external parties.\n\n\nQuestion 1: Could you specify the central theme of the content?\nAnswer: The central theme of the content is the importance of responsible and ethical conduct by employees in their interactions with external parties, with a focus on compliance with the company's policies and the avoidance of conflicts of interest. It emphasizes the value the company places on its employees as key assets and outlines the need for employees to seek guidance when in doubt about the legality or compliance of their actions. The content also highlights the company's commitment to coordinated communication with securities market professionals, shareholders, and the media through the Investor Relations Manager to ensure the provision of timely, accurate, and complete information.\n\n\nQuestion 2: What is the main focus of the content described?\nAnswer: The main focus of the content described is the company's employment practices, particularly regarding the ethical conduct and responsibilities of employees when dealing with external inquiries and ensuring the integrity of financial information. It emphasizes the importance of employees as valuable assets to the company's success and outlines the need for adherence to the company's Code of Conduct, especially in relation to coordinated communication with securities market professionals, shareholders, and the media through the designated Investor Relations Manager. The content also stresses the importance of vigilance against any financial inconsistencies, transactions without clear business purposes, and attempts to bypass standard review and approval procedures, all of which are crucial for maintaining the company's reputation.\n\n\nQuestion 3: Can you identify the primary subject covered in the content?\nAnswer: The primary subject covered in the content is the company's employment practices related to the ethical conduct and responsibilities of employees when responding to inquiries from securities market professionals, shareholders, and members of the media. It includes the policy of referring such inquiries to the Investor Relations Manager to ensure coordinated, accurate, and complete communication. The content also touches on the importance of making fair and accurate comments about competitors based on publicly available information, and it advises employees to seek guidance when in doubt about the legality or compliance with company policies. Additionally, it relates to the adherence to the company's Code of Conduct and the maintenance of the company's reputation through vigilance against financial inconsistencies and adherence to standard review and approval procedures.\n\n\nQuestion 4: What is the content primarily concerned with?\nAnswer: The content is primarily concerned with the company's employment practices and policies regarding the ethical conduct and responsibilities of employees when dealing with external inquiries from securities market professionals, shareholders, and the media. I